In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import sys
sys.path.append("../../psap_utils")
from psap_utils import find_best_params_average
from psap_utils import print_results_in_separate_lines
from metrics import weighted_balanced_accuracy
# 机器学习模型库
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier

# 其他工具库
import warnings
warnings.filterwarnings('ignore')
import re
import pickle
import os
import time
from datetime import datetime
from joblib import Parallel, delayed

# 评估和模型选择库
from sklearn.metrics import recall_score, precision_score, confusion_matrix, roc_auc_score, make_scorer, accuracy_score, f1_score, fbeta_score, matthews_corrcoef, balanced_accuracy_score, brier_score_loss, class_likelihood_ratios


In [2]:
with open('b_rac_splits.pkl', 'rb') as f:
    datasets = pickle.load(f)

In [46]:
scoring= {
    'acc': make_scorer(accuracy_score),
    #'auc' make_scorer(roc_auc_score, needs_proba=True, average='macro'),
    'mcc': make_scorer(matthews_corrcoef),
    #'wba': make_scorer(weighted_balanced_accuracy),
    'ba': make_scorer(balanced_accuracy_score),
    #'f1': make_scorer(f1_score)
}

In [58]:
models = [
    ("LR", LogisticRegression(max_iter=400), {
        'solver': ['newton-cg', 'lbfgs', 'sag'],
        'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
        'penalty': ['l2', 'none']
    }),
    ("SVM", SVC(probability=True), {
        'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000], 
        'kernel': ['linear', 'rbf', 'poly']
    }),
    ("RF", RandomForestClassifier(criterion='gini', min_samples_split=2, bootstrap=True), {
        'n_estimators': [50, 100, 150, 200], 
        'max_depth': [3, 4, 5]
    }),
    ("XGB", XGBClassifier(booster='gbtree'), {
        'n_estimators': [50, 100, 150, 200], 
        'max_depth': [3, 4, 5], 
        'learning_rate': [0.001, 0.01, 0.1]
    }),
    ("ANN", MLPClassifier(hidden_layer_sizes=(42, 21,), early_stopping=False), {
        'hidden_layer_sizes': [(42, 21,), (21, 14,), (42, 21, 14,)],
        'activation': ['tanh'],
        'solver': ['sgd', 'adam'],
        'alpha': [0.001, 0.01, 0.1],
        'learning_rate': ['adaptive'],
        'max_iter': [200, 300, 400]
    })]

In [6]:
all_best_params = {}

# Iterate over models
for model_name, model, params in models:
    print(f"Processing {model_name}...")
    
    # Call the function for finding the best parameters on average
    best_params = find_best_params_average(datasets=datasets, 
                                           model=model, 
                                           param_grid=params, 
                                           cv_folds=5,  # or another number if you prefer
                                           scoring=scoring)
    
    # Save the results associated with the model
    all_best_params[model_name] = best_params

    print_results_in_separate_lines(model_name,best_params)
    

Processing LR...


Processing datasets: 100%|████████████████████████████████████████████████████████| 10/10 [00:20<00:00,  2.09s/dataset]


Best parameters for LR:
  acc:
    best_params: {'C': 1000, 'penalty': 'l2', 'solver': 'lbfgs'}
    best_score: 0.81984496124031

  auc:
    best_params: {'C': 0.001, 'penalty': 'none', 'solver': 'newton-cg'}
    best_score: 0.9000560712813839

  mcc:
    best_params: {'C': 1000, 'penalty': 'l2', 'solver': 'lbfgs'}
    best_score: 0.6543402764980939

  wba:
    best_params: {'C': 0.001, 'penalty': 'none', 'solver': 'newton-cg'}
    best_score: 0.8070209619270081

  ba:
    best_params: {'C': 1000, 'penalty': 'l2', 'solver': 'lbfgs'}
    best_score: 0.8297667309837214

  f1:
    best_params: {'C': 0.001, 'penalty': 'none', 'solver': 'newton-cg'}
    best_score: 0.8282139692749058

Processing SVM...


Processing datasets: 100%|████████████████████████████████████████████████████████| 10/10 [04:25<00:00, 26.55s/dataset]


Best parameters for SVM:
  acc:
    best_params: {'C': 1000, 'kernel': 'linear'}
    best_score: 0.8158139534883722

  auc:
    best_params: {'C': 1000, 'kernel': 'linear'}
    best_score: 0.8969097644590288

  mcc:
    best_params: {'C': 10, 'kernel': 'linear'}
    best_score: 0.6748275694485353

  wba:
    best_params: {'C': 1000, 'kernel': 'linear'}
    best_score: 0.7927965425425756

  ba:
    best_params: {'C': 100, 'kernel': 'linear'}
    best_score: 0.8335938950604092

  f1:
    best_params: {'C': 1000, 'kernel': 'linear'}
    best_score: 0.8151472253899918

Processing RF...


...

Best parameters for RF:
  acc:
    best_params: {'max_depth': 5, 'n_estimators': 200}
    best_score: 0.8338501291989664

  auc:
    best_params: {'max_depth': 5, 'n_estimators': 200}
    best_score: 0.9243094284437918

  mcc:
    best_params: {'max_depth': 5, 'n_estimators': 200}
    best_score: 0.6944740367426914

  wba:
    best_params: {'max_depth': 5, 'n_estimators': 200}
    best_score: 0.8139472564933422

  ba:
    best_params: {'max_depth': 5, 'n_estimators': 200}
    best_score: 0.8484577008650648

  f1:
    best_params: {'max_depth': 5, 'n_estimators': 200}
    best_score: 0.836255262517865

Processing XGB...


Processing datasets: 100%|████████████████████████████████████████████████████████| 10/10 [00:48<00:00,  4.87s/dataset]


Best parameters for XGB:
  acc:
    best_params: {'learning_rate': 0.01, 'max_depth': 4, 'n_estimators': 100}
    best_score: 0.8499224806201552

  auc:
    best_params: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 50}
    best_score: 0.9264807373798936

  mcc:
    best_params: {'learning_rate': 0.01, 'max_depth': 4, 'n_estimators': 150}
    best_score: 0.7010341704114292

  wba:
    best_params: {'learning_rate': 0.01, 'max_depth': 4, 'n_estimators': 50}
    best_score: 0.8476624381544238

  ba:
    best_params: {'learning_rate': 0.01, 'max_depth': 4, 'n_estimators': 200}
    best_score: 0.8536397856208306

  f1:
    best_params: {'learning_rate': 0.01, 'max_depth': 4, 'n_estimators': 100}
    best_score: 0.8641692701976785

Processing ANN...


Processing datasets: 100%|████████████████████████████████████████████████████████| 10/10 [04:08<00:00, 24.87s/dataset]

Best parameters for ANN:
  acc:
    best_params: {'activation': 'tanh', 'alpha': 0.01, 'hidden_layer_sizes': (42, 21), 'learning_rate': 'adaptive', 'max_iter': 300, 'solver': 'sgd'}
    best_score: 0.8073385012919896

  auc:
    best_params: {'activation': 'tanh', 'alpha': 0.001, 'hidden_layer_sizes': (21, 14), 'learning_rate': 'adaptive', 'max_iter': 200, 'solver': 'adam'}
    best_score: 0.8785122621790166

  mcc:
    best_params: {'activation': 'tanh', 'alpha': 0.01, 'hidden_layer_sizes': (42, 21), 'learning_rate': 'adaptive', 'max_iter': 300, 'solver': 'sgd'}
    best_score: 0.6433043899179061

  wba:
    best_params: {'activation': 'tanh', 'alpha': 0.1, 'hidden_layer_sizes': (42, 21), 'learning_rate': 'adaptive', 'max_iter': 200, 'solver': 'adam'}
    best_score: 0.7916115331993059

  ba:
    best_params: {'activation': 'tanh', 'alpha': 0.01, 'hidden_layer_sizes': (42, 21), 'learning_rate': 'adaptive', 'max_iter': 300, 'solver': 'sgd'}
    best_score: 0.8221997532443748

  f1:
   

In [7]:
with open('b_rac_all_best_params.pkl', 'wb') as f:
    pickle.dump(all_best_params, f)

In [ ]:
with open('b_rac_all_best_params.pkl', 'rb') as f:
    all_best_params = pickle.load(f)

In [8]:
all_best_params

{'LR': {'acc': {'best_params': {'C': 1000, 'penalty': 'l2', 'solver': 'lbfgs'},
   'best_score': 0.81984496124031},
  'auc': {'best_params': {'C': 0.001,
    'penalty': 'none',
    'solver': 'newton-cg'},
   'best_score': 0.9000560712813839},
  'mcc': {'best_params': {'C': 1000, 'penalty': 'l2', 'solver': 'lbfgs'},
   'best_score': 0.6543402764980939},
  'wba': {'best_params': {'C': 0.001,
    'penalty': 'none',
    'solver': 'newton-cg'},
   'best_score': 0.8070209619270081},
  'ba': {'best_params': {'C': 1000, 'penalty': 'l2', 'solver': 'lbfgs'},
   'best_score': 0.8297667309837214},
  'f1': {'best_params': {'C': 0.001, 'penalty': 'none', 'solver': 'newton-cg'},
   'best_score': 0.8282139692749058}},
 'SVM': {'acc': {'best_params': {'C': 1000, 'kernel': 'linear'},
   'best_score': 0.8158139534883722},
  'auc': {'best_params': {'C': 1000, 'kernel': 'linear'},
   'best_score': 0.8969097644590288},
  'mcc': {'best_params': {'C': 10, 'kernel': 'linear'},
   'best_score': 0.67482756944853

In [30]:
unique_parameters = {}

# Extracting and deduplicating parameters
for model_name, metrics in all_best_params.items():
    for metric, content in metrics.items():
        params = content['best_params']
        if model_name not in unique_parameters:
            unique_parameters[model_name] = set()  # Initialize as a new set
        unique_parameters[model_name].add(frozenset(params.items()))  # Now you can safely add the items

In [23]:
unique_parameters

{'LR': {frozenset({('C', 1000), ('penalty', 'l2'), ('solver', 'lbfgs')}),
  frozenset({('C', 0.001), ('penalty', 'none'), ('solver', 'newton-cg')})},
 'SVM': {frozenset({('C', 10), ('kernel', 'linear')}),
  frozenset({('C', 100), ('kernel', 'linear')}),
  frozenset({('C', 1000), ('kernel', 'linear')})},
 'RF': {frozenset({('max_depth', 5), ('n_estimators', 200)})},
 'XGB': {frozenset({('learning_rate', 0.01),
             ('max_depth', 4),
             ('n_estimators', 150)}),
  frozenset({('learning_rate', 0.01),
             ('max_depth', 4),
             ('n_estimators', 100)}),
  frozenset({('learning_rate', 0.1), ('max_depth', 3), ('n_estimators', 50)}),
  frozenset({('learning_rate', 0.01),
             ('max_depth', 4),
             ('n_estimators', 200)}),
  frozenset({('learning_rate', 0.01),
             ('max_depth', 4),
             ('n_estimators', 50)})},
 'ANN': {frozenset({('activation', 'tanh'),
             ('alpha', 0.1),
             ('hidden_layer_sizes', (42, 21))

In [59]:
from sklearn.base import clone  # 用于复制模型实例

# 初始化字典，每个模型名对应一个空列表
configured_models = {model_name: [] for model_name, _, _ in models}

for model_name, model_instance, _ in models:
    # 检查确保有为该模型定义的唯一参数
    if model_name in unique_parameters:
        for param in unique_parameters[model_name]:
            # 复制原始模型实例以避免修改它
            model_copy = clone(model_instance)
            
            # 更新模型参数
            model_copy.set_params(**dict(param))  # 将 frozenset 转换为字典
            
            # 将配置过的模型实例添加到列表中
            configured_models[model_name].append(model_copy)

# 现在，configured_models 字典中的每个条目都是对应模型的配置实例列表


In [60]:
configured_models

{'LR': [LogisticRegression(C=1000, max_iter=400),
  LogisticRegression(C=0.001, max_iter=400, penalty='none', solver='newton-cg')],
 'SVM': [SVC(C=10, kernel='linear', probability=True),
  SVC(C=100, kernel='linear', probability=True),
  SVC(C=1000, kernel='linear', probability=True)],
 'RF': [RandomForestClassifier(max_depth=5, n_estimators=200)],
 'XGB': [XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
                colsample_bylevel=None, colsample_bynode=None,
                colsample_bytree=None, early_stopping_rounds=None,
                enable_categorical=False, eval_metric=None, feature_types=None,
                gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
                interaction_constraints=None, learning_rate=0.01, max_bin=None,
                max_cat_threshold=None, max_cat_to_onehot=None,
                max_delta_step=None, max_depth=4, max_leaves=None,
                min_child_weight=None, missing=nan, monotone_constraints=

In [ ]:
data = pd.read_csv('../../data/data_rac.csv')
y = data['RAC']
prevalence = y.sum()/y.shape[0]


In [70]:
results = {}

# 假设 datasets 是一个包含所有数据集的列表，每个数据集都是一个字典，包含 'X_train', 'y_train', 'X_test', 'y_test'
for model_name, model_list in configured_models.items():
    for index, model in enumerate(model_list):  # 添加索引以区分不同的模型实例
        unique_model_name = f"{model_name}: model{index + 1}"  # 为每个模型创建一个唯一的名称，如 "LR: 模型1"
        scores = {key: [] for key in scoring}

        for dataset in datasets:
            X_train, y_train, X_test, y_test = dataset['X_train'], dataset['y_train'], dataset['X_test'], dataset['y_test']

            # 训练模型并做出预测
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)

            if hasattr(model, "predict_proba"):
                y_proba = model.predict_proba(X_test)[:, 1]  # 获取属于正类的概率
            else:
                y_proba = None

            for name, scorer in scoring.items():
                if 'auc' in name and y_proba is not None:
                    score_value = scorer._score_func(y_test, y_proba, **scorer._kwargs)
                elif 
                else:
                    score_value = scorer._score_func(y_test, y_pred, **scorer._kwargs)
                scores[name].append(score_value)

        # 计算每个评分指标的平均值
        average_scores = {key: np.mean(value) for key, value in scores.items()}

        # 将结果保存在 'results' 字典中，使用模型的唯一名称作为键
        results[unique_model_name] = average_scores

# 'results' 字典现在包含每个模型实例的评分，每个实例都有一个唯一的名称


In [71]:
results

{'LR: 模型1': {'acc': 0.8212074303405572,
  'auc': 0.9026479903034096,
  'mcc': 0.6565601267770683,
  'wba': 0.8076673443853613,
  'ba': 0.8309303644041289,
  'f1': 0.828930406139046},
 'LR: 模型2': {'acc': 0.8199690402476779,
  'auc': 0.9033420003127933,
  'mcc': 0.6511752099851924,
  'wba': 0.8081782400166823,
  'ba': 0.828435838285893,
  'f1': 0.8292465717864139},
 'SVM: 模型1': {'acc': 0.8071207430340557,
  'auc': 0.8992160619330625,
  'mcc': 0.6726009713998958,
  'wba': 0.7753167031592118,
  'ba': 0.8299587503909915,
  'f1': 0.7972027200261401},
 'SVM: 模型2': {'acc': 0.8139318885448917,
  'auc': 0.9005605841413825,
  'mcc': 0.6716250468421313,
  'wba': 0.7870855489521427,
  'ba': 0.8332098451673444,
  'f1': 0.8095462817698154},
 'SVM: 模型3': {'acc': 0.8181114551083593,
  'auc': 0.9010175555208008,
  'mcc': 0.6730177795180806,
  'wba': 0.7939370242936085,
  'ba': 0.8354707538317172,
  'f1': 0.8163902527508322},
 'RF: 模型1': {'acc': 0.8301857585139321,
  'auc': 0.9295169299343135,
  'mcc': 0

In [61]:
results = {}

# 假设 datasets 是一个包含所有数据集的列表，每个数据集都是一个字典，包含 'X_train', 'y_train', 'X_test', 'y_test'
for model_name, model_list in configured_models.items():
    results[model_name] = []
    for model in model_list:
        # 存储当前模型的所有评分
        scores = {key: [] for key in scoring}
        
        for dataset in datasets:
            X_train, y_train, X_test, y_test = dataset['X_train'], dataset['y_train'], dataset['X_test'], dataset['y_test']
            
            # 在训练集上训练模型
            model.fit(X_train, y_train)
            
            # 在测试集上评估模型
            predictions = model.predict(X_test)
            probabilities = model.predict_proba(X_test)[:,1] 

            for name, scorer in scoring.items():
                if 'auc' in name and probabilities is not None:
                    # 对于需要预测概率的评分指标，如 AUC
                    score_value = scorer._score_func(y_test, probabilities, **scorer._kwargs)
                else:
                    # 对于不需要预测概率的评分指标
                    score_value = scorer._score_func(y_test, predictions, **scorer._kwargs)
                scores[name].append(score_value)
        
        # 计算每个指标的平均分数
        average_scores = {key: np.mean(value) for key, value in scores.items()}
        results[model_name].append(average_scores)

# 最后，results 字典包含了每种模型在测试集上的平均评分


KeyboardInterrupt: 

In [55]:
datasets[0]['y_test'].shape

(646,)

In [41]:
all_average_scores = {}
for model_name, model in configured_models:
    average_scores = {}
    for dataset in tqdm(datasets, desc="Processing datasets", unit="dataset"):
        X_train, y_train, X_test, y_test = dataset['X_train'], dataset['y_train'], dataset['X_test'], dataset['y_test']

        # 训练模型
        model.fit(X_train, y_train)
            # 对于每个指标，计算得分并将其添加到相应的列表中
        for metric, scorer in scoring.items():
            if metric == 'auc':
                # 对于需要概率的评分指标，我们使用 predict_proba
                y_proba = lr.predict_proba(X_test)[:1]
                score = scorer(lr, X_test, y_test)  # 使用概率来计算分数
            else:
                # 对于其他指标，我们使用预测的类别
                y_pred = lr.predict(X_test)
                score = scorer(lr, X_test, y_test)  # 使用预测来计算分数

            scores[metric].append(score)

    # 计算每个指标的平均分数
    average_scores = {metric: np.mean(score_values) for metric, score_values in scores.items()}

Processing datasets:   0%|                                                                 | 0/10 [00:00<?, ?dataset/s]


AttributeError: 'str' object has no attribute 'fit'

In [49]:
for configured_model in configured_models:
    print(configured_model)

LR
SVM
RF
XGB
ANN


In [51]:
from sklearn.metrics import accuracy_score, roc_auc_score, matthews_corrcoef, f1_score, balanced_accuracy_score
import numpy as np

# 准备一个字典来收集每个模型的结果
results = {}

# 遍历每种类型的模型配置
for model_type, model_list in configured_models.items():
    print(f"Processing {model_type}...")

    # 对于每种类型的每个模型配置，我们将其结果存储在此列表中
    type_results = []

    for model in model_list:
        # 用于存储当前模型配置的各种评分
        scores = {key: [] for key in scoring}

        # 假设 'datasets' 是您的多个数据集的列表
        for dataset in datasets:
            X_train, y_train, X_test, y_test = dataset['X_train'], dataset['y_train'], dataset['X_test'], dataset['y_test']

            # 训练模型并进行评分
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)

            # 如果评分机制需要概率或决策函数，我们这里处理
            if 'auc' in scoring:
                # 有些模型方法可能没有 'predict_proba'
                if hasattr(model, 'predict_proba'):
                    y_score = model.predict_proba(X_test)[:, 1]
                elif hasattr(model, 'decision_function'):
                    y_score = model.decision_function(X_test)
                else:
                    raise RuntimeError(f"Model {model_type} does not have 'predict_proba' or 'decision_function' method required for 'auc' scoring.")

                scores['auc'].append(roc_auc_score(y_test, y_score))

            # 计算其他指标
            for metric, scorer in scoring.items():
                if metric != 'auc':  # 我们已经处理过了 AUC
                    score = scorer(model, y_test, y_pred)
                    scores[metric].append(score)

        # 计算平均分数并存储结果
        avg_scores = {metric: np.mean(values) for metric, values in scores.items()}
        type_results.append(avg_scores)

    # 将结果存储到主结果字典中
    results[model_type] = type_results

# 输出结果
for model_type, type_results in results.items():
    print(f"Results for {model_type}:")
    for config_results in type_results:
        print(config_results)


Processing LR...


ValueError: Expected 2D array, got 1D array instead:
array=[0 1 0 1 1 1 1 0 0 0 1 1 0 1 0 1 1 1 0 1 1 1 0 1 1 0 0 0 0 0 1 0 1 1 0 1 1
 0 1 0 0 1 1 1 1 1 1 1 0 1 1 0 0 0 0 0 1 1 0 1 1 0 1 1 0 0 0 0 0 1 1 0 1 1
 1 1 1 1 0 1 1 0 1 0 1 1 1 0 1 1 1 0 1 0 0 0 1 0 0 1 1 0 0 0 1 0 1 0 0 0 0
 0 0 0 0 0 0 1 1 1 1 0 0 0 0 1 1 0 1 1 0 1 0 1 1 1 1 1 0 0 0 1 0 1 1 1 1 0
 1 0 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 0 0 1 0 0 1 1 1 1 0 1 1 1 0 1 0 1
 0 1 1 1 0 1 0 1 1 1 0 1 1 1 1 1 1 1 0 1 1 1 1 0 0 1 1 1 0 0 0 0 1 1 1 0 0
 1 1 1 1 1 1 1 0 0 1 0 0 0 0 1 1 1 1 0 0 0 0 0 1 1 1 0 0 0 1 0 1 0 1 1 1 1
 0 1 1 1 1 1 0 1 1 1 0 0 1 1 1 1 1 1 1 1 1 0 1 1 0 0 1 1 1 1 1 0 1 1 1 1 1
 1 0 1 1 0 0 0 1 1 0 1 1 1 0 1 1 0 1 1 1 1 1 0 1 0 1 0 0 0 1 1 1 0 1 0 1 1
 0 0 1 0 1 0 0 0 1 0 1 1 0 1 1 1 0 0 0 1 0 0 1 0 1 0 1 0 1 0 0 1 1 0 0 0 0
 1 1 1 1 1 0 1 1 1 0 0 1 0 1 0 1 1 1 1 0 1 0 0 1 0 1 0 0 1 0 1 1 0 1 1 0 0
 0 1 1 1 1 0 1 1 0 1 0 1 1 0 1 1 1 0 0 0 0 0 0 0 1 1 0 1 0 1 1 1 0 1 1 0 0
 0 1 0 0 1 0 1 1 1 1 1 1 1 1 0 1 1 1 1 1 